# Laboratorio 2

In [2]:
import numpy as np
import random


1. Implementar una función en Python que genere un arreglo de números aleatorios entre 0  y 9 (1.0 puntos)

Requerimientos:
*   Entradas: tamaño del arreglo
*   Salida: arreglo de numpy con elementos de 32 bits

In [3]:

arr_size = 5
arreglo_original = np.random.randint(0,9,arr_size,dtype=np.int32)
arreglo_salida = np.random.randint(0,1,arr_size,dtype=np.int8) #Esto es solo para verificar que es correcto el método para crear arreglos con 0s, ya que igual se pasará a crear un neuvo arreglo después
print(arreglo_original)
print(arreglo_salida)

[8 7 8 2 2]
[0 0 0 0 0]



2.   Implementar una función en Python que detecte los números pares de un arreglo (1.0 puntos)

Requerimientos:



*   Entrada: arreglo de numpy con elementos de 32 bits
*   Salida: arreglo de numpy con elementos de 8 bits
*   Cuando el número es par el valor del arreglo de salida en la posición de ese elemento será 0, y cuando es impar será 1.




In [3]:
import ctypes

def sacar_pares(arreglo_original):
    arr_size = len(arreglo_original)
    arreglo_salida = np.random.randint(0,1,arr_size,dtype=np.int8)
    for n in range(arr_size):
        if(arreglo_original[n]%2==0):
            arreglo_salida[n]=0
        else:
            arreglo_salida[n]=1
    print("En Python: ")
    print(arreglo_salida)

print(arreglo_original)
sacar_pares(arreglo_original)


[2 2 4 2 7]
En Python: 
[0 0 0 0 1]



3.   Implementar una función en C que detecte los números pares de un arreglo (2.0 puntos)

Requerimientos:



*   Entradas: 
    + Puntero a los elementos de entrada enteros de 32 bits
    + Cantidad de elementos
    + Puntero a los elementos de salida enteros de 8 bits





In [4]:
%%file arr_c.c
void func_array_c(int *arreglo_original, int cant, char *arreglo_salida){
    for(int i=0; i<cant; i++){
        if(arreglo_original[i]%2==0){
            arreglo_salida[i]='0'-48;
        }
        else{
            arreglo_salida[i]='1'-48;
        }
    }
}

Writing arr_c.c




4.   Generar el *object file* y la *shared library* (1.0 puntos)



In [5]:
# crear el object file
! gcc -c -Wall -Werror -fpic arr_c.c

# crear la shared library
! gcc -shared arr_c.o -o arr_c.so




5.   Hacer una función que vincule la librería dinámica de C con Python (1.0 puntos)



In [6]:
import ctypes

def llamar_funcion_c():
    # indicar la ruta de la shared library
    libfile = './arr_c.so'

    # cargar la shared library
    lib = ctypes.CDLL(libfile)

    # tipo de dato de los argumentos
    lib.func_array_c.argtypes = [np.ctypeslib.ndpointer(dtype=np.int32),ctypes.c_int,np.ctypeslib.ndpointer(dtype=np.int8)]
    
    # funcion configurada
    return lib.func_array_c

In [7]:
func_c = llamar_funcion_c()



6.   Envolver la función implementada en C para que cumpla con los mismos requerimientos que la función implementada en Pyhon (1.0 puntos)



In [8]:
def funcion_envuelta_c(arreglo_original):
    arr_size = len(arreglo_original)
    arreglo_salida_c = np.random.randint(0,1,arr_size,dtype=np.int8)
    func_c(arreglo_original, arr_size, arreglo_salida_c)
    return arreglo_salida_c

print(arreglo_original)
funcion_envuelta_c(arreglo_original)

[2 2 4 2 7]


array([0, 0, 0, 0, 1], dtype=int8)

7.   Implementar una función en ASM que detecte los números pares de un arreglo (4.0 puntos)

Requerimientos:



*   Entradas: 
    + Puntero a los elementos de entrada enteros de 32 bits
    + Cantidad de elementos
    + Puntero a los elementos de salida enteros de 8 bits

In [19]:
%%file func_array_asm.asm
global func_array_asm
    section .text
;*arreglo_original => rdi
;arr_size => rsi
;*arreglo_salida => rdx

func_array_asm:
    mov eax, 2
    mov rcx, rdx
    
_iteracion:
    xor rdx, rdx
    idiv edi
    cmp edx, 0
    je _esPar
    cmp edx, 1
    je _esImpar
    _numeroEvaluado:
    add rcx, 4
    add rdi, 4
    dec rsi
    cmp rsi, 0
    jne _iteracion

_done:
    mov rdx, rcx
    ret

_esPar:
    mov ecx, 0
    jmp _numeroEvaluado
_esImpar:
    mov ecx, 1
    jmp _numeroEvaluado

Overwriting func_array_asm.asm


8.   Generar el *object file* y la *shared library* (1.0 puntos)

In [20]:
%%file arr_asm.c
extern void func_array_asm(int *arreglo_original, int cant, short int *arreglo_salida);

Overwriting arr_asm.c


In [21]:
# Oject file de NASM!
! nasm -f elf64 func_array_asm.asm -o func_array_asm.o

# crear el object file
! gcc -c -Wall -Werror -fpic arr_asm.c

# crear la shared library
! gcc -shared func_array_asm.o arr_asm.c -o arr_asm.so

9.   Hacer una función que vincule la librería dinámica de ASM con Python (1.0 puntos)

In [57]:
import ctypes

def llamar_funcion_asm():
    # indicar la ruta de la shared library
    libfile = './arr_asm.so'

    # cargar la shared library
    lib2 = ctypes.CDLL(libfile)

    # tipo de dato de los argumentos
    lib2.func_array_asm.argtypes = [np.ctypeslib.ndpointer(dtype=np.int32),ctypes.c_int,np.ctypeslib.ndpointer(dtype=np.int8)]
    
    # funcion configurada
    return lib2.func_array_asm

In [58]:
func_asm = llamar_funcion_asm()

10.   Envolver la función implementada en ASM para que cumpla con los mismos requerimientos que la función implementada en Pyhon (1.0 puntos)


In [59]:
def funcion_envuelta_asm(arreglo_original):
    arr_size = len(arreglo_original)
    arreglo_salida_asm = np.random.randint(0,1,arr_size,dtype=np.int8)
    func_asm(arreglo_original, arr_size, arreglo_salida_asm)   
    return arreglo_salida_asm

print(arreglo_original)
funcion_envuelta_asm(arreglo_original)

[7 2 1 2 0]


array([0, 0, 0, 0, 0], dtype=int8)



11.   Validar que las tres funciones dan el mismo resultado para un tamaño de `16384` elementos. (1.0 puntos)



In [1]:
arr_size = 16384
arreglo_original = np.random.randint(0,9,arr_size,dtype=np.int32)
print(arreglo_original)

sacar_pares(arreglo_original)
funcion_envuelta_c(arreglo_original)
funcion_envuelta_asm(arreglo_original)


NameError: name 'np' is not defined




12.   Realice mediciones de tiempo para las funciones implementadas. Las mediciones serán  para la lista de tamaños `[32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]`. Para cada tamaño realice 100 mediciones y filtre sus mediciones con la función `filtro_mediana` con una ventana de 7. Emplee el método `perf_counter` del módulo `time`. (1.0 puntos)



In [ ]:
import statistics

def filtro_mediana(sign, ventana):
    signc = []
    tam = len(sign)
    offs = int((ventana-1)/2)
    for i in range(tam):
        inicio = i - offs if (i - offs > 0) else i
        fin = i + offs if (i + offs < tam) else tam
        signc.append(statistics.median(sign[inicio:fin]))
    return signc



13.   Presente las siguientes gráficas (4.0 puntos):
     +   Tiempo vs Tamaño
        - Una gráfica en la que estén Python, C y ASM
        - Una gráfica en la que estén solo C y ASM
     +   Speedup vs Tamaño
        - Una gráfica del speedup de C y ASM respecto de Python
        - Una gráfica del speedup de ASM respecto de C




In [ ]:
import matplotlib.pyplot as plt